In [7]:
import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm

from model import Model


class Net(nn.Module):
    def __init__(self, num_class, pretrained_path):
        super(Net, self).__init__()

        # encoder
        self.f = Model().f
        # classifier
        self.fc = nn.Linear(2048, num_class, bias=True)
        self.load_state_dict(torch.load(pretrained_path, map_location='cpu'), strict=False)

    def forward(self, x):
        x = self.f(x)
        feature = torch.flatten(x, start_dim=1)
        out = self.fc(feature)
        return out


# train or test for one epoch
def train_val(net, data_loader, train_optimizer=None):
    is_train = train_optimizer is not None
    net.train() if is_train else net.eval()

    total_loss, total_correct_1, total_correct_5, total_num = 0.0, 0.0, 0.0, 0
    data_bar = tqdm(data_loader, leave=False)
    
    with (torch.enable_grad() if is_train else torch.no_grad()):
        for data, target in data_bar:
            data, target = data.cuda(non_blocking=True), target.cuda(non_blocking=True)
            out = net(data)
            loss = loss_criterion(out, target)

            if is_train:
                train_optimizer.zero_grad()
                loss.backward()
                train_optimizer.step()

            total_num += data.size(0)
            total_loss += loss.item() * data.size(0)
            prediction = torch.argsort(out, dim=-1, descending=True)
            total_correct_1 += torch.sum((prediction[:, 0:1] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()
            total_correct_5 += torch.sum((prediction[:, 0:5] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()
        
            if is_train:
                data_bar.set_description(('Train Epoch : {}/{} Loss: {:.4f} ACC@1: {:.2f}% ACC@5: {:.2f}%'
                                     .format(epoch, epochs, total_loss / total_num,
                                             total_correct_1 / total_num * 100, total_correct_5 / total_num * 100)))
            else:
                data_bar.set_description(('Test Loss: {:.4f} ACC@1: {:.2f}% ACC@5: {:.2f}%'.format(total_loss / total_num,
                                             total_correct_1 / total_num * 100, total_correct_5 / total_num * 100)))


    return total_loss / total_num, total_correct_1 / total_num * 100, total_correct_5 / total_num * 100

In [10]:
#변형된 데이터를 이용한 훈련

from torchvision import transforms
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import utils

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

batch_size, epochs = 128, 100
model_path = 'results_batch32/model.pth'

train_data = CIFAR10(root='.data/', train=True, transform=train_transform, download=True)

train_loader, valid_loader = utils.create_datasets(batch_size, train_data)
# model setup and optimizer config


model = Net(num_class=len(train_data.classes), pretrained_path=model_path).cuda()
for param in model.f.parameters():
    param.requires_grad = False

optimizer = optim.Adam(model.fc.parameters(), lr=1e-3, weight_decay=1e-6)
loss_criterion = nn.CrossEntropyLoss()
results = {'train_loss': [], 'train_acc@1': [], 'train_acc@5': [], 'valid_acc@1': []}

best_acc = 0
for epoch in range(1, epochs + 1):
    train_loss, train_acc_1, train_acc_5 = train_val(model, train_loader, optimizer)
    _, valid_acc_1, _ = train_val(model, valid_loader)
    results['train_loss'].append(train_loss)
    results['train_acc@1'].append(train_acc_1)
    results['train_acc@5'].append(train_acc_5)
    results['valid_acc@1'].append(valid_acc_1)
    
    if best_acc<valid_acc_1:
        best_epoch = epoch
        best_acc = valid_acc_1
        torch.save(model.state_dict(), 'results_batch32/linear_model.pth')
        
        
    data_frame = pd.DataFrame(data=results, index=range(1, epoch + 1))
    data_frame.to_csv('results_batch32/linear_statistics.csv', index_label='epoch')

print(best_epoch)

Files already downloaded and verified


86


In [11]:
from torchvision import transforms

batch_size = 128
loss_criterion = nn.CrossEntropyLoss()

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

test_data = CIFAR10(root='.data/', train=False, transform=test_transform, download=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=16, pin_memory=True)

test_results = {'test_loss': [], 'test_acc@1': [], 'test_acc@5': []}
model_path = 'results_batch32/linear_model.pth'
model = Net(num_class=len(test_data.classes), pretrained_path=model_path).cuda()
test_loss, test_acc_1, test_acc_5 = train_val(model, test_loader, None)
test_results['test_loss'].append(test_loss)
test_results['test_acc@1'].append(test_acc_1)
test_results['test_acc@5'].append(test_acc_5)
print(test_results)

Files already downloaded and verified


{'test_loss': [0.455409330034256], 'test_acc@1': [83.99], 'test_acc@5': [99.41]}
